In [1]:
from urllib.request import urlopen

In [34]:
import re

In [35]:
import pandas as pd

In [36]:
url = "https://air.gov.ge/en/reports_page?station=ORN06%2CORN08%2CORN03%2CBTUM%2CORN05%2CAGMS%2CVRKT%2CORN01%2CKZBG%2CTSRT%2CORN07%2CRST18%2CORN02%2CKUTS%2CORN04&report_type=monthly&date_from=2025-01"
page = urlopen(url)

In [37]:
html = page.read().decode('utf-8')

In [39]:
def decodeURL(url:str) -> BeautifulSoup:
    '''
    Takes url, returns decoded html
    '''
    html = urlopen(url).read().decode('utf-8')
    return BeautifulSoup(html, 'html.parser')

In [79]:
STATIONCODES = ["ORN06", "ORN08", "ORN03", "BTUM", "ORN05", "AGMS", "VRKT", "ORN01", "KZBG", "TSRT", "ORN07", "RST18", "ORN02", "KUTS", "ORN04"]

In [382]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime, timedelta
from tqdm import tqdm

In [422]:
def decodeURL(url:str) -> BeautifulSoup:
    '''
    Takes url, returns decoded html
    '''
    html = urlopen(url).read().decode('utf-8')
    return BeautifulSoup(html, 'html.parser')

def readTables(tables:list[str]):
    data = []
    counter = 0
    for i in range(len(tables)):
        table = tables[i].find_all('td')
        text = tables[i].find('th').get_text()
        header = sortHeader(text)
        for j in range(len(table)):
            if j%7== 0:
                data.extend(header)
            data.append(table[j].get_text())
    return data

def sortHeader(text:str) -> list[str]:
    text = text.replace('\n', '')
    splittext = text.split(',')
    name = splittext[0].replace(' ', '')
    codedate = splittext[1].split('  ')
    code = codedate[0].replace(' ', '')
    date = codedate[1]
    return [name, code, date]

def get_monthly_dates(start="2016-08", end="2025-01"):
    start_date = datetime.strptime(start, "%Y-%m")
    end_date = datetime.today().replace(day=1)  # First day of the current month
    months = []
    
    while start_date <= end_date:
        months.append(start_date.strftime("%Y-%m"))
        start_date += timedelta(days=32)
        start_date = start_date.replace(day=1)
    return months

def createDF(url:str) -> pd.DataFrame:
    soup = decodeURL(url)
    data = readTables(soup.find_all('table'))
    return pd.DataFrame(np.array(data, dtype=object).reshape(len(data)//10, 10), 
                 columns = ['name', 'code', 'year-month', 'day', 'NO<sub>2', 'SO<sub>2', 'PM2.5', 'PM10', 'O<sub>3', 'CO'])
def run_scraper():
    url_base = "https://air.gov.ge/en/reports_page?station=ORN06%2CORN08%2CORN03%2CBTUM%2CORN05%2CAGMS%2CVRKT%2CORN01%2CKZBG%2CTSRT%2CORN07%2CRST18%2CORN02%2CKUTS%2CORN04&report_type=monthly&date_from="
    url_dates = get_monthly_dates()
    final_data = pd.DataFrame(columns = ['name', 'code', 'year-month', 'day', 'NO<sub>2', 'SO<sub>2', 'PM2.5', 'PM10', 'O<sub>3', 'CO'])
    for date in tqdm(url_dates):
        url = url_base+date
        temp_data = createDF(url)
        final_data = pd.concat([final_data, temp_data], ignore_index=True)
    # final_data = final_data.reset_index()
    final_data.to_csv('AirData.csv', sep=',')
    return final_data

In [423]:
data = run_scraper()

100%|██████████| 103/103 [02:46<00:00,  1.62s/it]
